In [ ]:
import pickle
from google.colab import drive

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import os

In [ ]:
# Google dirve에 연결

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PATH
PATH_train='/content/drive/MyDrive/montage_project/train/'
PATH_train_montage = '/content/drive/MyDrive/montage_project/train/montage/'
PATH_train_json = '/content/drive/MyDrive/montage_project/train/H/'

PATH_train_resized='/content/drive/MyDrive/montage_project/train/resized/'

In [ ]:
PATH_test='/content/drive/MyDrive/montage_project/test/'
PATH_test_montage = '/content/drive/MyDrive/montage_project/test/montage/'
PATH_test_json = '/content/drive/MyDrive/montage_project/test/H/'


In [ ]:
# 라벨 설정
class_names_age = ['20~35','25~50','50']
class_names_gender = ['M','F']
class_names_face = ['계란형','긴형','둥근형','마름모형','사각형','역삼각형']
class_names_hairstyle = ['곱슬머리','직모(생머리)','나머지']
class_names_eyebrows =['각진형','둥근형','물결형','직선형','분류없음']

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 불러오기

In [ ]:
import gzip
# 2. 전처리 + 헤어스타일 라벨 3개 + 증강

## train 불러오기!
with gzip.open(PATH_train+'dic_train_label2.pickle','rb') as f:
  dic_train_label = pickle.load(f)

print(len(dic_train_label),dic_train_label.keys())

6 dict_keys(['train_img_ar', 'train_age_label', 'train_gender_label', 'train_face_label', 'train_hairstyle_label', 'train_eyebrows_label'])


In [ ]:
dic_train_label['train_img_ar'][0].shape

(224, 224, 3)

In [ ]:
len(dic_train_label['train_img_ar']),len(dic_train_label['train_age_label']),len(dic_train_label['train_gender_label']),len(dic_train_label['train_hairstyle_label'])

(32089, 32089, 32089, 32089)

In [ ]:
import gzip
# 2. 전처리 + 헤어스타일 라벨 3개 + 증강

## test 불러오기!
with gzip.open(PATH_test+'dic_test_label2.pickle','rb') as f:
  dic_test_label = pickle.load(f)

print(len(dic_test_label),dic_test_label.keys())

6 dict_keys(['test_img_ar', 'test_age_label', 'test_gender_label', 'test_face_label', 'test_hairstyle_label', 'test_eyebrows_label'])


In [ ]:
dic_test_label['test_img_ar'][0].shape

(224, 224, 3)

In [ ]:
len(dic_test_label['test_img_ar']),len(dic_test_label['test_age_label']),len(dic_test_label['test_gender_label']),len(dic_test_label['test_hairstyle_label'])

(1010, 1010, 1010, 1010)

## Tensorflow

In [ ]:
import tensorflow as tf

tf.__version__

'2.8.2'

텐서플로우 2.0 버전에서는 Keras가 기본 탑재가 되었고   
공식사이트에서도 Keras를 사용하기 때문에 저희도 Keras 위주로 진행을 하겠습니다.  
(keras 로 코딩을 하는게 훨씬 쉽습니다.)

In [ ]:
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

# CNN - Age




In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
train_input = train_img_ar
test_input = test_img_ar

train_target = train_age_label
test_target =test_age_label

# 정규화
train_input, test_input = train_input / 255.0, test_input / 255.0

In [ ]:
train_input.shape,train_target.shape,test_input.shape,test_target.shape

((8071, 224, 224, 3), (8071,), (1010, 224, 224, 3), (1010,))

In [ ]:
set(train_age_label)

{0, 1, 2}

In [ ]:
#모델 구축
model = models.Sequential()
## filtering layer
# input = 180*135
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224,224,3)))   #64=필터의 개수, 3,3은 필터의 크기
model.add(layers.MaxPooling2D((2, 2)))  #필터의 개수 채널 입력할 필요없다!! 앞의 이미지 따라가니까, activation 없다->비선형 통과한게 아니니까
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))   #11*11 을 2*2패딩 적용하면 ? 5*5
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

## classification layer
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
# output = 3개!
model.add(layers.Dense(3, activation='softmax'))

#모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 256)       295168    
                                                                 
 flatten (Flatten)           (None, 692224)            0

In [ ]:
# 학습
model.fit(train_input, train_target, epochs=5,validation_split=0.2,verbose=1)

Epoch 1/5
202/202 [==============================] - 900s 4s/step - loss: 0.5954 - accuracy: 0.7988 - val_loss: 0.8264 - val_accuracy: 0.6217
Epoch 2/5
202/202 [==============================] - 897s 4s/step - loss: 0.2456 - accuracy: 0.8937 - val_loss: 0.5195 - val_accuracy: 0.7771
Epoch 3/5
202/202 [==============================] - 897s 4s/step - loss: 0.1913 - accuracy: 0.9167 - val_loss: 0.6000 - val_accuracy: 0.7009
Epoch 4/5
202/202 [==============================] - 895s 4s/step - loss: 0.1787 - accuracy: 0.9266 - val_loss: 0.4548 - val_accuracy: 0.8272
Epoch 5/5
202/202 [==============================] - 873s 4s/step - loss: 0.1516 - accuracy: 0.9370 - val_loss: 0.4433 - val_accuracy: 0.8248


In [ ]:
# 시험
test_loss, test_acc = model.evaluate(test_input, test_target, verbose=2)
print(test_acc)

32/32 - 35s - loss: 0.2939 - accuracy: 0.9178 - 35s/epoch - 1s/step
0.9178217649459839


# CNN-gender

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(train_img_ar,
                                                                      train_gender_label,
                                                                      shuffle=True, 
                                                                      stratify=train_age_label, 
                                                                      random_state=34)

# 정규화
train_input, test_input = train_input / 255.0, test_input / 255.0

In [ ]:
train_input.shape,train_target.shape,test_input.shape,test_target.shape

((6053, 180, 135, 3), (6053,), (2018, 180, 135, 3), (2018,))

In [ ]:
set(train_Data['gender'])

{'F', 'M'}

In [ ]:
set(train_gender_label)

{0, 1}

In [ ]:
#모델 구축
model = models.Sequential()
## filtering layer
# input = 180*135
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(180,135,3)))   #64=필터의 개수, 3,3은 필터의 크기
model.add(layers.MaxPooling2D((2, 2)))  #필터의 개수 채널 입력할 필요없다!! 앞의 이미지 따라가니까, activation 없다->비선형 통과한게 아니니까
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))   #11*11 을 2*2패딩 적용하면 ? 5*5
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

## classification layer
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
# output = 2개!
model.add(layers.Dense(2, activation='softmax'))

#모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 178, 133, 64)      1792      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 89, 66, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 87, 64, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 43, 32, 128)      0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 41, 30, 256)       295168    
                                                                 
 flatten_3 (Flatten)         (None, 314880)           

In [ ]:
# 학습
model.fit(train_input, train_target, epochs=5,validation_split=0.2,verbose=1)

Epoch 1/5
152/152 [==============================] - 244s 2s/step - loss: 0.3116 - accuracy: 0.9025 - val_loss: 0.1489 - val_accuracy: 0.9463
Epoch 2/5
152/152 [==============================] - 244s 2s/step - loss: 0.1188 - accuracy: 0.9566 - val_loss: 0.0987 - val_accuracy: 0.9661
Epoch 3/5
152/152 [==============================] - 245s 2s/step - loss: 0.0912 - accuracy: 0.9667 - val_loss: 0.0732 - val_accuracy: 0.9777
Epoch 4/5
152/152 [==============================] - 248s 2s/step - loss: 0.0676 - accuracy: 0.9715 - val_loss: 0.1079 - val_accuracy: 0.9744
Epoch 5/5
152/152 [==============================] - 249s 2s/step - loss: 0.0649 - accuracy: 0.9738 - val_loss: 0.0764 - val_accuracy: 0.9761


In [ ]:
# 시험
test_loss, test_acc = model.evaluate(test_input, test_target, verbose=2)
print(test_acc)

64/64 - 27s - loss: 0.0619 - accuracy: 0.9797 - 27s/epoch - 416ms/step
0.9796828627586365


# CNN-face

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(train_img_ar,
                                                                      train_face_label,
                                                                      shuffle=True, 
                                                                      stratify=train_face_label, 
                                                                      random_state=34)

# 정규화
train_input, test_input = train_input / 255.0, test_input / 255.0

In [ ]:
train_input.shape,train_target.shape,test_input.shape,test_target.shape

((6053, 180, 135, 3), (6053,), (2018, 180, 135, 3), (2018,))

In [ ]:
set(dic_Data['face'])

{'계란형', '긴형', '둥근형', '마름모형', '사각형', '역삼각형'}

In [ ]:
set(Data_face_label)

{0, 1, 2, 3, 4, 5}

In [ ]:
#모델 구축
model = models.Sequential()
## filtering layer
# input = 180*135
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(180,135,3)))   #64=필터의 개수, 3,3은 필터의 크기
model.add(layers.MaxPooling2D((2, 2)))  #필터의 개수 채널 입력할 필요없다!! 앞의 이미지 따라가니까, activation 없다->비선형 통과한게 아니니까
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))   #11*11 을 2*2패딩 적용하면 ? 5*5
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

## classification layer
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
# output
model.add(layers.Dense(6, activation='softmax'))

#모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 178, 133, 64)      1792      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 89, 66, 64)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 87, 64, 128)       73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 43, 32, 128)      0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 41, 30, 256)       295168    
                                                                 
 flatten_4 (Flatten)         (None, 314880)           

In [ ]:
# 학습
model.fit(train_input, train_target, epochs=5,validation_split=0.2,verbose=1)

Epoch 1/5
152/152 [==============================] - 248s 2s/step - loss: 1.4973 - accuracy: 0.3567 - val_loss: 1.3522 - val_accuracy: 0.4005
Epoch 2/5
152/152 [==============================] - 248s 2s/step - loss: 1.3270 - accuracy: 0.4329 - val_loss: 1.2764 - val_accuracy: 0.4814
Epoch 3/5
152/152 [==============================] - 252s 2s/step - loss: 1.2435 - accuracy: 0.4806 - val_loss: 1.2576 - val_accuracy: 0.4690
Epoch 4/5
152/152 [==============================] - 346s 2s/step - loss: 1.1892 - accuracy: 0.5103 - val_loss: 1.2311 - val_accuracy: 0.4889
Epoch 5/5
152/152 [==============================] - 242s 2s/step - loss: 1.1428 - accuracy: 0.5310 - val_loss: 1.2506 - val_accuracy: 0.4814


In [ ]:
# 시험
test_loss, test_acc = model.evaluate(test_input, test_target, verbose=2)
print(test_acc)

64/64 - 24s - loss: 1.2350 - accuracy: 0.4911 - 24s/epoch - 375ms/step
0.49108028411865234


# CNN-hairstyle

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(train_img_ar,
                                                                      train_hairstyle_label,
                                                                      shuffle=True, 
                                                                      stratify=train_hairstyle_label, 
                                                                      random_state=34)

# 정규화
train_input, test_input = train_input / 255.0, test_input / 255.0

In [ ]:
train_input.shape,train_target.shape,test_input.shape,test_target.shape

((6053, 180, 135, 3), (6053,), (2018, 180, 135, 3), (2018,))

In [ ]:
set(dic_Data['hairstyle'])

{'곱슬머리', '스포츠형', '웨이브형', '직모(생머리)', '탈모형'}

In [ ]:
set(Data_hairstyle_label)

{0, 1, 2, 3, 4}

In [ ]:
#모델 구축
model = models.Sequential()
## filtering layer
# input = 180*135
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(180,135,3)))   #64=필터의 개수, 3,3은 필터의 크기
model.add(layers.MaxPooling2D((2, 2)))  #필터의 개수 채널 입력할 필요없다!! 앞의 이미지 따라가니까, activation 없다->비선형 통과한게 아니니까
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))   #11*11 을 2*2패딩 적용하면 ? 5*5
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

## classification layer
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
# output
model.add(layers.Dense(5, activation='softmax'))

#모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 178, 133, 64)      1792      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 89, 66, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 87, 64, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 43, 32, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 41, 30, 256)       295168    
                                                                 
 flatten_5 (Flatten)         (None, 314880)           

In [ ]:
# 학습
model.fit(train_input, train_target, epochs=5,validation_split=0.2,verbose=1)

Epoch 1/5
152/152 [==============================] - 236s 2s/step - loss: 1.0869 - accuracy: 0.5549 - val_loss: 0.9256 - val_accuracy: 0.5714
Epoch 2/5
152/152 [==============================] - 236s 2s/step - loss: 0.8783 - accuracy: 0.6010 - val_loss: 0.8647 - val_accuracy: 0.6350
Epoch 3/5
152/152 [==============================] - 249s 2s/step - loss: 0.8283 - accuracy: 0.6262 - val_loss: 0.8400 - val_accuracy: 0.6309
Epoch 4/5
152/152 [==============================] - 237s 2s/step - loss: 0.7921 - accuracy: 0.6499 - val_loss: 0.8358 - val_accuracy: 0.6416
Epoch 5/5
152/152 [==============================] - 235s 2s/step - loss: 0.7551 - accuracy: 0.6656 - val_loss: 0.7965 - val_accuracy: 0.6466


In [ ]:
# 시험
test_loss, test_acc = model.evaluate(test_input, test_target, verbose=2)
print(test_acc)

64/64 - 24s - loss: 1.2350 - accuracy: 0.4911 - 24s/epoch - 375ms/step
0.49108028411865234


# RESNET50 -Age


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
train_input = train_img_ar
test_input = test_img_ar

train_target = train_age_label
test_target =test_age_label


from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input,
                                                                      train_target,
                                                                      shuffle=True, 
                                                                      stratify=train_target, 
                                                                      random_state=34)




In [ ]:
train_input = train_input.astype('float32')
val_input=val_input.astype('float32')
test_input=test_input.astype('float32')

In [ ]:
def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0)

In [ ]:
zero_mean_img = zero_mean(train_input)

In [ ]:
train_input -= zero_mean_img
val_input -= zero_mean_img
test_input -= zero_mean_img

In [ ]:
train_input.shape,train_target.shape,val_input.shape,val_target.shape

((6053, 224, 224, 3), (6053,), (2018, 224, 224, 3), (2018,))

In [ ]:
test_input.shape,test_target.shape

((1010, 224, 224, 3), (1010,))

In [ ]:
set(train_age_label)

{0, 1, 2}

In [ ]:
## 일부 재학습 모델
base_model = ResNet50(include_top=False, input_shape = (224, 224,3), weights = 'imagenet')

for layer in base_model.layers[:-10]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 49,278,851
Trainable params: 30,156,803
Non-tra

In [ ]:
# 학습
model.fit(train_input, train_target, epochs = 5, batch_size= 125, validation_data=(val_input, val_target))

Epoch 1/5
49/49 [==============================] - 742s 15s/step - loss: 1.0215 - accuracy: 0.7780 - val_loss: 0.2923 - val_accuracy: 0.8860
Epoch 2/5
49/49 [==============================] - 754s 15s/step - loss: 0.2488 - accuracy: 0.9000 - val_loss: 0.2434 - val_accuracy: 0.8944
Epoch 3/5
49/49 [==============================] - 810s 17s/step - loss: 0.1956 - accuracy: 0.9197 - val_loss: 0.2307 - val_accuracy: 0.9024
Epoch 4/5
49/49 [==============================] - 780s 16s/step - loss: 0.1613 - accuracy: 0.9338 - val_loss: 0.2540 - val_accuracy: 0.8940
Epoch 5/5
49/49 [==============================] - 794s 16s/step - loss: 0.1298 - accuracy: 0.9456 - val_loss: 0.2379 - val_accuracy: 0.8999


In [ ]:
# 시험
model.evaluate(test_input, test_target)

32/32 [==============================] - 89s 3s/step - loss: 0.4365 - accuracy: 0.9069


[0.43645957112312317, 0.906930685043335]

# RESNET50 - gender


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
type(dic_train_label['train_img_ar']),type(dic_test_label['test_img_ar'])

(numpy.ndarray, numpy.ndarray)

In [ ]:
len(dic_train_label['train_img_ar']),len(dic_test_label['test_img_ar'])

(32089, 1010)

In [ ]:
dic_train_label['train_img_ar'][0].shape,dic_test_label['test_img_ar'][0].shape

((224, 224, 3), (224, 224, 3))

In [ ]:
type(dic_train_label['train_age_label']),type(dic_test_label['test_age_label'])

(numpy.ndarray, numpy.ndarray)

In [ ]:
len(dic_train_label['train_age_label']),len(dic_test_label['test_age_label'])

(32089, 1010)

In [ ]:
dic_train_label['train_age_label'][10000],dic_test_label['test_age_label'][1000]

(0, 2)

In [ ]:
train_input = dic_train_label['train_img_ar']
test_input = dic_test_label['test_img_ar']

train_target = dic_train_label['train_gender_label']
test_target =dic_test_label['test_gender_label']


from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input,
                                                                      train_target,
                                                                      shuffle=True, 
                                                                      stratify=train_target, 
                                                                      random_state=34)

In [ ]:
train_input = train_input.astype('float32')
val_input=val_input.astype('float32')
test_input=test_input.astype('float32')

In [ ]:
def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0)

In [ ]:
zero_mean_img = zero_mean(train_input)
zero_mean_img.shape

(224, 224, 3)

In [ ]:
train_input -= zero_mean_img
val_input -= zero_mean_img
test_input -= zero_mean_img

UFuncTypeError: ignored

In [ ]:
train_input.shape,train_target.shape,val_input.shape,val_target.shape

((24066, 224, 224, 3), (24066,), (8023, 224, 224, 3), (8023,))

In [ ]:
test_input.shape,test_target.shape

((1010, 224, 224, 3), (1010,))

In [ ]:
set(train_target)

{0, 1}

In [ ]:
## 일부 재학습 모델
base_model = ResNet50(include_top=False, input_shape = (224, 224,3), weights = 'imagenet')

for layer in base_model.layers[:-10]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 49,278,594
Trainable params: 30,156,546
Non-tra

In [ ]:
# 학습
model.fit(train_input, train_target, epochs = 5, batch_size= 125, validation_data=(val_input, val_target))

## fine-tune
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_input, train_target, epochs = 5, validation_data=(val_input, val_target), 
                   batch_size= 125)

# 시험
model.evaluate(test_input, test_target)

Epoch 1/5
193/193 [==============================] - 2834s 15s/step - loss: 0.4224 - accuracy: 0.8795 - val_loss: 0.1330 - val_accuracy: 0.9511
Epoch 2/5
193/193 [==============================] - 2839s 15s/step - loss: 0.1245 - accuracy: 0.9525 - val_loss: 0.1116 - val_accuracy: 0.9576
Epoch 3/5
193/193 [==============================] - 2838s 15s/step - loss: 0.0995 - accuracy: 0.9628 - val_loss: 0.1034 - val_accuracy: 0.9619
Epoch 4/5
193/193 [==============================] - 2850s 15s/step - loss: 0.0765 - accuracy: 0.9708 - val_loss: 0.1060 - val_accuracy: 0.9612
Epoch 5/5
181/193 [===========================>..] - ETA: 2:17 - loss: 0.0563 - accuracy: 0.9781

In [ ]:
## fine-tune
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_input, train_target, epochs = 5, validation_data=(val_input, val_target), 
                   batch_size= 125)

In [ ]:
# 시험
model.evaluate(test_input, test_target)

32/32 [==============================] - 89s 3s/step - loss: 0.4365 - accuracy: 0.9069


[0.43645957112312317, 0.906930685043335]

# RESNET50 - hairstyle


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models #Tensorflow에 있는 Keras 함수들 호출하기

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
type(dic_train_label['train_img_ar']),type(dic_test_label['test_img_ar'])

(numpy.ndarray, numpy.ndarray)

In [ ]:
len(dic_train_label['train_img_ar']),len(dic_test_label['test_img_ar'])

(32089, 1010)

In [ ]:
dic_train_label['train_img_ar'][0].shape,dic_test_label['test_img_ar'][0].shape

((224, 224, 3), (224, 224, 3))

In [ ]:
type(dic_train_label['train_hairstyle_label']),type(dic_test_label['test_hairstyle_label'])

(numpy.ndarray, numpy.ndarray)

In [ ]:
len(dic_train_label['train_hairstyle_label']),len(dic_test_label['test_hairstyle_label'])

(32089, 1010)

In [ ]:
dic_train_label['train_hairstyle_label'][10000],dic_test_label['test_hairstyle_label'][1000]

(0, 1)

In [ ]:
set(dic_train_label['train_hairstyle_label'])

{0, 1, 2}

In [ ]:
train_input = dic_train_label['train_img_ar']
test_input = dic_test_label['test_img_ar']

train_target = dic_train_label['train_hairstyle_label']
test_target =dic_test_label['test_hairstyle_label']

In [ ]:
# sparse categorical crossentropy VS 
# sparse categorical crossentropy 역시 멀티클래스 분류에 사용되는 손실함수입니다. 이름에 sparse 하나가 추가되었죠. 그러면 sparse categorical crossentropy가 사용되는 경우는 언제일까요? 바로 라벨이 0, 1, 2, 3, 4와 같이 정수의 형태로 제공될 때입니다. 
# categorical crossentropy는 분류해야할 클래스가 3개 이상인 경우, 즉 멀티클래스 분류에 사용됩니다. 라벨이 [0,0,1,0,0], [1,0,0,0,0], [0,0,0,1,0]과 같이 one-hot 형태로 제공될 때 사용됩니다.
# categorical crossentropy + one_hot 
# 원핫 인코딩 (50000,1) -> (50000,10) => 카테고리컬
train_target = tf.keras.utils.to_categorical(train_target, 3)
test_target = tf.keras.utils.to_categorical(test_target, 3)
train_target.shape, test_target.shape

((32089, 3), (1010, 3))

In [ ]:
train_target[0]

array([0., 1., 0.], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input,
                                                                      train_target,
                                                                      shuffle=True)

In [ ]:
train_input.shape,train_target.shape,val_input.shape,val_target.shape

((24066, 224, 224, 3), (24066, 3), (8023, 224, 224, 3), (8023, 3))

In [ ]:
test_input.shape,test_target.shape

((1010, 224, 224, 3), (1010, 3))

In [ ]:
train_target.shape

(24066, 3)

In [ ]:
## 일부 재학습 모델
base_model = ResNet50(include_top=False, input_shape = (224, 224,3), weights = 'imagenet')

for layer in base_model.layers[:-10]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 49,278,851
Trainable params: 30,156,803
Non-tra

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0)

In [ ]:
# 학습
# 2. 전처리 + 헤어스타일 라벨 3개 + 증강(train = 32089개) + zeromean 불가능
# 카테고리컬

model.fit(train_input, train_target, epochs = 15, batch_size= 125, validation_data=(val_input, val_target),callbacks=[callback])

Epoch 1/15
193/193 [==============================] - 2844s 15s/step - loss: 1.0453 - accuracy: 0.5684 - val_loss: 0.7961 - val_accuracy: 0.6382 - lr: 1.0000e-04
Epoch 2/15
193/193 [==============================] - 2841s 15s/step - loss: 0.7826 - accuracy: 0.6481 - val_loss: 0.7608 - val_accuracy: 0.6519 - lr: 1.0000e-04
Epoch 3/15
193/193 [==============================] - 2844s 15s/step - loss: 0.7301 - accuracy: 0.6709 - val_loss: 0.7453 - val_accuracy: 0.6601 - lr: 1.0000e-04
Epoch 4/15
193/193 [==============================] - 2885s 15s/step - loss: 0.6821 - accuracy: 0.6966 - val_loss: 0.7344 - val_accuracy: 0.6804 - lr: 1.0000e-04
Epoch 5/15
193/193 [==============================] - 2909s 15s/step - loss: 0.6323 - accuracy: 0.7174 - val_loss: 0.7347 - val_accuracy: 0.6759 - lr: 1.0000e-04
Epoch 6/15
193/193 [==============================] - 2939s 15s/step - loss: 0.5823 - accuracy: 0.7403 - val_loss: 0.7589 - val_accuracy: 0.6691 - lr: 1.0000e-04
Epoch 7/15
193/193 [========

In [ ]:
## fine-tune
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_input, train_target, epochs = 15, validation_data=(val_input, val_target), batch_size= 125,callbacks=[callback])

Epoch 1/15
193/193 [==============================] - 8500s 44s/step - loss: 0.9833 - accuracy: 0.5849 - val_loss: 0.7907 - val_accuracy: 0.6378 - lr: 1.0000e-04
Epoch 2/15


In [ ]:
# 시험
model.evaluate(test_input, test_target)